# Partitioning Thresholds in QuASAr

This notebook derives theoretical thresholds for when QuASAr's planner switches simulation methods and introduces circuit partitions.  The estimates are based solely on the calibrated cost model and therefore provide insight into the planner's decisions independent of empirical runtime measurements.

## Methodology

QuASAr incrementally grows a fragment of the input circuit and uses a `MethodSelector` to estimate the cost of running that fragment on the available simulation backends.  Whenever the cheapest backend changes for the extended fragment, the planner finalises the current partition and starts a new one.  The `CostEstimator` exposes analytic runtime and memory models for:

* Dense statevector simulation (`Backend.STATEVECTOR`)
* Stabilizer tableau methods (`Backend.TABLEAU`)
* Matrix product states (`Backend.MPS`)
* Decision diagrams (`Backend.DECISION_DIAGRAM`)

The following sections compare these cost models to highlight cross-over points where a different backend becomes cheaper.  These cross-overs act as theoretical thresholds for partitioning.  Conversion costs between backends are estimated using primitives such as boundary-to-boundary (B2B) extraction and local windows (LW).

In [ ]:
from quasar.cost import CostEstimator, Backend
import numpy as np
import matplotlib.pyplot as plt

estimator = CostEstimator()

### Tableau vs. Statevector
The tableau simulator handles Clifford-only circuits in \(O(n^2)\) time, whereas dense statevector simulation scales with \(2^n\).  The plot below assumes ten Clifford gates per qubit and shows where the tableau model becomes cheaper.

In [ ]:
n_range = np.arange(1, 15)
sv_times = []
tab_times = []
for n in n_range:
    g = 10 * n
    sv_times.append(estimator.statevector(n, g, 0, 0).time)
    tab_times.append(estimator.tableau(n, g).time)
sv_times = np.array(sv_times)
tab_times = np.array(tab_times)
# first qubit count where tableau is cheaper
crossover = n_range[np.argmax(tab_times < sv_times)]
plt.plot(n_range, sv_times, label="Statevector")
plt.plot(n_range, tab_times, label="Tableau")
plt.axvline(crossover, color="k", linestyle="--", label=f"threshold={crossover}")
plt.xlabel("qubits")
plt.ylabel("estimated runtime")
plt.title("Clifford fragment cost")
plt.legend()
plt.show()
print(f"Tableau becomes cheaper from {crossover} qubits onwards")

### Conversion overhead context

Statevector and tableau costs cross at different qubit counts, but the planner
rarely switches backends without extracting a boundary first. Quantifying the
extra conversions clarifies when it is still worthwhile to introduce a
partition.

In [ ]:
n_range = np.arange(4, 33)
stay_cost = []
partition_cost = []
for n in n_range:
    boundary = min(n, 6)
    rank = min(2 ** boundary, 32)
    frontier = boundary
    prefix_1q = 2 * n
    prefix_2q = 2 * max(n - 1, 0)
    clifford_1q = 6 * n
    clifford_2q = 6 * max(n - 1, 0)

    stay = estimator.statevector(
        n,
        prefix_1q + clifford_1q,
        prefix_2q + clifford_2q,
        0,
    ).time
    prefix = estimator.statevector(n, prefix_1q, prefix_2q, 0).time
    tableau = estimator.tableau(n, clifford_1q + clifford_2q).time
    sv_to_tab = estimator.conversion(
        Backend.STATEVECTOR,
        Backend.TABLEAU,
        num_qubits=boundary,
        rank=rank,
        frontier=frontier,
    ).cost.time
    tab_to_sv = estimator.conversion(
        Backend.TABLEAU,
        Backend.STATEVECTOR,
        num_qubits=boundary,
        rank=rank,
        frontier=frontier,
    ).cost.time

    stay_cost.append(stay)
    partition_cost.append(prefix + sv_to_tab + tableau + tab_to_sv)

stay_cost = np.array(stay_cost)
partition_cost = np.array(partition_cost)
plt.plot(n_range, stay_cost, label="Statevector only")
plt.plot(n_range, partition_cost, label="Partition with conversions")
cheaper = np.where(partition_cost < stay_cost)[0]
if cheaper.size:
    threshold = n_range[cheaper[0]]
    plt.axvline(threshold, color="k", linestyle="--", label=f"threshold={threshold}")
    print(f"Partitioned execution becomes cheaper from {threshold} qubits onwards")
else:
    threshold = None
    print("Partitioned execution is never cheaper in this range")
plt.xlabel("qubits")
plt.ylabel("estimated runtime")
plt.title("Cost of staying on statevector vs. switching to tableau")
plt.legend()
plt.show()

### Scenario assumptions

- **Gate density.** The circuit starts with two layers of non-Clifford two-qubit
  entanglers and single-qubit phase gates (modelled as `2n` single- and
  `2(n-1)` two-qubit operations). This prefix motivates keeping a dense
  statevector before partitioning. The remaining block contributes
  `6n` single- and `6(n-1)` two-qubit Clifford gates, matching the tableau cost
  estimate.
- **Boundary size.** We convert a six-qubit interface (or the full register if
  it is smaller) representative of a narrow cut between a dense and a Clifford
  partition.
- **Rank/frontier.** The conversion uses a Schmidt-rank cap of 32 and sets the
  decision-diagram frontier equal to the boundary width, reflecting modest
  entanglement at the cut. Adjust these parameters to explore deeper prefixes or
  wider partitions.

### MPS vs. Statevector
For local circuits the matrix-product-state (MPS) backend scales with the bond dimension \(\chi\) instead of \(2^n\).  We assume a linear nearest-neighbour circuit with five single- and two-qubit gates per qubit and \(\chi=4\).

In [ ]:
n_range = np.arange(2, 15)
sv_times = []
mps_times = []
for n in n_range:
    num_1q = 5 * n
    num_2q = 5 * (n - 1)
    sv_times.append(estimator.statevector(n, num_1q, num_2q, 0).time)
    mps_times.append(estimator.mps(n, num_1q, num_2q, chi=4, svd=True).time)
sv_times = np.array(sv_times)
mps_times = np.array(mps_times)
crossover = n_range[np.argmax(mps_times < sv_times)]
plt.plot(n_range, sv_times, label="Statevector")
plt.plot(n_range, mps_times, label="MPS")
plt.axvline(crossover, color="k", linestyle="--", label=f"threshold={crossover}")
plt.xlabel("qubits")
plt.ylabel("estimated runtime")
plt.title("Local circuit cost")
plt.legend()
plt.show()
print(f"MPS becomes cheaper from {crossover} qubits onwards")

### Decision Diagram vs. Statevector
Decision diagrams excel on sparse states.  Here we ignore sparsity heuristics and simply compare the cost models for a circuit with ten gates per qubit.

In [ ]:
n_range = np.arange(1, 20)
sv_times = []
dd_times = []
for n in n_range:
    g = 10 * n
    sv_times.append(estimator.statevector(n, g, 0, 0).time)
    dd_times.append(estimator.decision_diagram(num_gates=g, frontier=n).time)
sv_times = np.array(sv_times)
dd_times = np.array(dd_times)
plt.plot(n_range, sv_times, label="Statevector")
plt.plot(n_range, dd_times, label="Decision diagram")
plt.xlabel("qubits")
plt.ylabel("estimated runtime")
plt.title("Sparse circuit cost")
plt.legend()
plt.show()

### Conversion Primitive Selection
When switching between backends, the planner chooses the cheapest conversion primitive.  The example below converts a boundary of size \(q\) from a statevector to a decision diagram and shows which primitive minimises the cost.

In [ ]:
qs = np.arange(1, 10)
primitives = []
costs = []
for q in qs:
    conv = estimator.conversion(Backend.STATEVECTOR, Backend.DECISION_DIAGRAM,
                                num_qubits=q, rank=2**q, frontier=q)
    primitives.append(conv.primitive)
    costs.append(conv.cost.time)
plt.plot(qs, costs, marker='o')
for q, p, c in zip(qs, primitives, costs):
    plt.text(q, c, p, ha='center', va='bottom')
plt.xlabel('boundary size q')
plt.ylabel('conversion time')
plt.title('Conversion primitive by boundary size')
plt.show()